## Recupero lista di aziende dell'S&P 500

Prima di tutto si va a cercare la pagina source.

Con `ALT + CMD +U` si guarda la source e si valuta quello che è necessario prelevare

In [9]:
import bs4 as bs
import pickle
import requests

In [10]:
def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    # Paritamo la riga 1 così ignoriamo quella con i titoli delle varie colonne
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip().replace('.','-') # strip serve per rimuovere il \n
        tickers.append(ticker)
    
    with open('sp500ticker.pickle', 'wb') as f:
        pickle.dump(tickers, f)
    
    # print(tickers)
    return tickers

In [11]:
save_sp500_tickers()
print('Ok')

Ok


## Recupero informazioni di ogni azienda S&P 500

In [6]:
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web

In [12]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open('sp500ticker.pickle', 'rb') as f:
            tickers = pickle.load(f)
    
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    
    start = dt.datetime(2000,1,1)
    end = dt.datetime(2019,12,31)

    for ticker in tickers:
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            # print('Already exist {}'.format(ticker))
            pass

In [13]:
get_data_from_yahoo()